# Upper sets 

Upper sets are a special construction relevant to preorders. In order to compute these, we'll review and implement aspects of power sets, preorders, and monotone maps along the way.

In [ ]:
using Catlab.CategoricalAlgebra.FinSets, Catlab.CategoricalAlgebra
using DataStructures: SortedSet
using Test

## Power Sets
We can construct a powerset of a given set by considering all possible subsets.
Specifying a subset of a set `S` is precisely the same data as providing a function from `S` into the two-element set `yesno`:
{"Nope, I'm not in the subset", "Yes, I'm in the subset"}

In Catlab, a `FinSet` of size `n` is represented by integers `1..n`. We can do our own bookkeeping to keep track of what the indices correspond to, i.e.:
- 1 = "Nope, I'm not in the subset"
- 2 = "Yes, I'm in the subset"

Likewise, let's encode a set `goods` that we could trade (`{sheep, wheat, gold}`) by the finite set `{1,2,3}`.

Now we can represent the subset {sheep, gold} as a function from `goods` to `yesno`

In [ ]:
yesno = FinSet(2)  # {no, yes}
goods = FinSet(3)  # {sheep, wheat, gold}

sheep_gold = FinFunction([2,1,2], goods, yesno)

We often care about all possible subsets of a set, e.g. all possible combinations of materials used in a construction project. For a set of size `n`, there are `2ⁿ` such subsets, which we can enumerate with the following function: 

In [ ]:
"""This function can compute all elements of the power set"""
function powerset(s::FinSet)::Vector{SortedSet{Int}}
  result = [SortedSet{Int}()]  # start with empty subset
  for index in s  # consider subsets that have `index` as a member
    newres = SortedSet{Int}[]
    for res in result
      push!(newres, res ∪ [index])
    end
    append!(result, newres)
  end
  return result
end

for n in 1:5
  @test length(powerset(FinSet(n))) == 2^n
end

There are many different ways to represent to these subsets on a computer. One obvious way is to directly store the elements in a collection datatype, such as `Vector` or `Set`. However, as we saw earlier, subsets can *also* be represented as functions to `FinSet(2)`, i.e. `yesno`. 

In [ ]:
# Exercise 1: Create the associated FinFunction to `FinSet(2)` for a particular subset.
function subset_to_map(s::FinSet, subset::SortedSet{Int})::FinFunction
  # TODO
end

@test subset_to_map(FinSet(3), SortedSet([1,2,3])) == FinFunction([2,2,2])
@test subset_to_map(FinSet(2), SortedSet([2])) == FinFunction([1,2])
@test_throws ErrorException subset_to_map(FinSet(3), SortedSet([4]))
@test_throws ErrorException subset_to_map(FinSet(3), SortedSet([0]))

In [ ]:
# Optional exercise 1.5: And create the inverse function
function map_to_subset(f::FinFunction)::SortedSet{Int}
  # TODO
end

@test map_to_subset(FinFunction([2,2,2])) == SortedSet([1,2,3])
@test map_to_subset(FinFunction([1,1,1,1,1,1,1,1], FinSet(2))) == SortedSet{Int}()
@test map_to_subset(FinFunction([1,1,1,1,1,1,1,2])) == SortedSet([8])
@test_throws ErrorException map_to_subset(FinFunction([1,2,3], FinSet(3), FinSet(3)))

This encoding of subsets as functions has its merits is not particularly space efficient. An alternative encoding allows us to represent a subset with a *single integer*. We can do this because there is a canonical ordering on the subsets of an ordered finite set. For example:

Index | Explicit | Function to 2
----- | -------- | ------------
1     | { }      |  {no, no, no}
2     | {1}      | {yes, no, no}
3     | {2}      | {no, yes, no}
4     | {1,2}    | {yes, yes, no}
5     | {3}      | {no, no, yes}
6     | {1,3}    | {yes, no, yes}
7     | {2,3}    | {no, yes, yes}
8     | {1,2,3}  | {yes, yes, yes}

Therefore, subsets can be identified with their location within a particular vector containing all possible subsets.

In [ ]:
# Exercise 2: Assign a unique number to each subset. 

# Remember, for each element in the original set, we have associated a "no" or a "yes".
# This is reminiscent of a certain kind of numbering system.
function subset_to_int(subset::SortedSet{Int})::Int
  # TODO
end

subset_to_int(SortedSet([1])) == 2
subset_to_int(SortedSet([1,2,3])) == 8

for i in 1:5
  subset_indices = [subset_to_int(ss) for ss in powerset(FinSet(i))]
  @test sort(subset_indices) == collect(1:2^i)
end

In [ ]:
# Optional exercise 2.5: Write the inverse function. 
# The functions `bitstring(UInt16(i::Int))` we can save some time.
function int_to_subset(i::Int)::SortedSet{Int}
  # TODO
end

for i in 1:16
  @test i == subset_to_int(int_to_subset(i))
end

# Preorders
Preorders characterize _reflexive_, _transitive_ relations. Many real-life relations are characterized by these properties, e.g. hierarchy, reachability, preferability. We characteristically represent this relation with the `≤` symbol. Computationally, we'll represent preorders as relations, i.e. subsets of `S × S`. To check whether `a ≤ᵣ b`, we check if `(a,b) ∈ r`.

With this preferability relation applied to our goods, we might declare that `wheat ≤ sheep ≤ gold`, i.e. `2 ≤ 1` and `1 ≤ 3`. The preorder laws demand that other things are related, e.g. that gold is at least as desirable as wheat (`2 ≤ 3` by transitivity) and that sheep are at least as desirable as sheep (`2 ≤ 2` reflexivity). Our Julia constructor for `Preorder` will automatically add these required relations.

In [ ]:
struct Preorder
  carrier:: FinSet
  relation :: SortedSet{Pair{Int,Int}}
  """Construct valid preorder by taking reflexive/transitive closure"""
  function Preorder(carrier:: FinSet, rel :: Vector{Pair{Int,Int}})
    for (a,b) ∈ rel
      a ∈ carrier && b ∈ carrier || error("relation element not in carrier set")
    end
    relation = SortedSet(rel)
    for (a, b, c) in Iterators.product(carrier, carrier, carrier)
      if ((a => b) ∈ relation) && ((b => c) ∈ relation)
        push!(relation, a => c) # enforces relation is transitive
      end
    end
    for i in carrier
      push!(relation, i => i) # enforces that relation is reflexive
    end
    return new(carrier, relation)
  end
end

The preorder `Bool` has two elements, `true`, and `false`, and one nontrivial element in its relation: `false ≤ true`. This and many other examples are declared below. Another example to highlight is a set of products (`{berry, sword, helmet, potion}`) that we could purchase with our goods, ordered by desirability: suppose that swords and potions are more desirable than helmets, which are more desirable than berries.

In [ ]:
# Example preorders
goods = Preorder(FinSet(3), [2=>1, 1=>3])
products = Preorder(FinSet(4), [1=>3, 3=>2, 3=>4])

empty = Preorder(FinSet(0), Pair{Int,Int}[])
bool = Preorder(FinSet(2), [1=>2])
diamond = Preorder(FinSet(4), [1=>2,1=>3,2=>4,3=>4])
discrete = Preorder(FinSet(4),Pair{Int,Int}[])
full = Preorder(FinSet(4),[1=>2,1=>3,1=>4,2=>1,2=>1,3=>1,4=>1])

# Monotone maps
Monotone maps are analogous to functions, but respecting preorder structure.
- Monotonicity: `x ≤ y ⟹ f(x) ≤ f(y)`

In the context of montone maps between goods and products, we can view this as a kind of bartering trade, where for each of our goods we select a product that it can be exchanged for.

In [ ]:
struct Monotone_map
  domain::Preorder
  codomain::Preorder
  mapping::FinFunction
  function Monotone_map(domain::Preorder, cod::Preorder, mapping::FinFunction)
    ((dom(mapping) == domain.carrier) && (codom(mapping) == cod.carrier)
    ) || error("mapping mismatch")
    return new(domain,cod,mapping)
  end
end

In many cases, this monotonicity constraint characterizes a certain type of sanity (precisely the sanity that prevents us from trading a quarter for a penny). 

Suppose that we were proposed a trade that exchanges our gold for berries, and our other goods for swords. This is certainly a valid function of finite sets, but it would not make sense for our (hard-earned) gold to be exchanged for something that is not as desirable as what our less valuable wheat was exchanged for. The following exercise requires you to automate this check.

In [ ]:
# Exercise 3: Check if a map is monotone
function is_monotone(mm::Monotone_map)::Bool
  # todo
end

exchange_1 = Monotone_map(goods, products, FinFunction([3,1,4], FinSet(3), FinSet(4)))
@test is_monotone(exchange_1)
@test !is_monotone(Monotone_map(goods, products, FinFunction([2,2,1], FinSet(3), FinSet(4))))
@test is_monotone(Monotone_map(goods, products, FinFunction([4,4,4], FinSet(3), FinSet(4))))


One could use the products obtained to trade again for newer products. This induces a monotone map between the original goods and the newer products.

In [ ]:
# Exercise 4: write a function to compose monotone maps
function compose(m1::Monotone_map, m2::Monotone_map)::Monotone_map
  # TODO
end

bronze_silver = Preorder(FinSet(2), [1=>2]) # {bronze, silver}, isomorphic to bool
exchange_2 = Monotone_map(products, bronze_silver, FinFunction([1,2,2,2], FinSet(4), FinSet(2)))
m1m2 = compose(exchange_1, exchange2)
@test m1m2.mapping == FinFunction([2,1,2])
@test is_monotone(m1m2) # composition of monotones is a monotone

# Upper sets

An upper set is a subset of a preorder such that the inclusion of an element `x` requires that, for each `x ≤ y`, that `y` be included in the set, too. Imagining a numeric value being assigned to each element of the preorder (respecting `≤`), an upper set represents all elements above a particular threshold. For no such assignment of values would `{wheat, sheep}` be the only elements above some threshold (given that gold's value must be equal to or exceed the others); thus, this would not be a valid upper set.

In [ ]:
function uppersets(s::Preorder)::Vector{SortedSet{Int}}
  result = [SortedSet{Int}()] # start with empty set
  for index in s.carrier
    newres = SortedSet{Int}[]
    for res in result
      if index ∉ res
        push!(newres, res ∪ [j for (i,j) ∈ s.relation if i==index])
      end
    end
    append!(result, newres)
    unique!(result)
  end
  return result
end

We saw that maps into the set `yesno` had a special significance for sets. Likewise, maps into `Bool` from preorders are signficant. The data to specify one is identical to a map into `yesno`; however, there is an additional requirement in order for this map to be a monotone map.

In [ ]:
# Exercise 5: Enumerate monotone maps to bool by filtering maps to 2
function monotone_maps_to_bool(p::Preorder)::Vector{Monotone_map}
  pset_maps = ["""TODO""" for s in powerset(p.carrier)]
  return """TODO"""
end

bool_mms = monotone_maps_to_bool(bool) # monotone maps from bool to bool
@test [m.mapping.func for m in bool_mms] == [[1,1],[1,2],[2,2]]

These monotone maps to `Bool` correspond directly with upper sets, for any preorder.

In [ ]:
preorders = [empty, goods, products, bool, diamond, discrete, full]

for p in preorders
  @test length(uppersets(p)) == length(monotone_maps_to_bool(p))
end

Thus, any sequence of (rational) trades that culminates in a trade into the silver/bronze preorder (isomorphic to `Bool`) reveals something about the set of goods at the start of the trade sequence: the elements which ultimately are mapped to silver form an upper set.  